In [1019]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.nn import  functional as F
import numpy as np


In [1020]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.485, 0.456, 0.406), std=(0.229, 0.224, 0.223))
])

In [1021]:
epsilons = [.05, .1, .15, .2, .25, .3]
alexnet = models.alexnet(pretrained = True)

In [1022]:
D = torchvision.datasets.ImageFolder('/Users/pawel/Desktop/panda',transform = preprocess)
data = torch.utils.data.DataLoader(D)


In [1023]:
def attack(img,eps, data_gradient):
    data_gradient_sign = data_gradient.sign()
    new_img = img + data_gradient_sign*eps
    new_img = torch.clamp(new_img, 0, 1)
    return new_img, data_gradient*eps

In [1024]:
def test(model, eps, data):
    pandas = []
    distorted_pandas = []
    attack_maps = []
    for panda in data:
        panda = panda[0]
        pandas.append(panda)
        panda.requires_grad = True
        output = model(panda)
        init_pred = torch.max(output,dim=1)
        if 388!=init_pred[1]:
            print("sad")
            distorted_pandas.append(0)
            attack_maps.append(0)
            continue

        desired_output = torch.zeros(1000)

        desired_output[388]=1

        loss = F.mse_loss(output[0],desired_output)

        model.zero_grad()

        loss.backward()
        
        panda_grad = panda.grad.data
        distorted_panda, attack_map = attack(panda,eps,panda_grad)
        output = torch.max(model(distorted_panda),dim=1)
        #print(output[1][0])
        if output[1] == 388:
            print("well, it failed")
            distorted_pandas.append(0)
            attack_maps.append(0)
            continue
        print("succes")
        distorted_pandas.append(distorted_panda)
        attack_maps.append(attack_map)
    return pandas, distorted_pandas, attack_maps

In [ ]:
pandas_zoo = []
distorted_pandas_zoo = []
attack_maps_zoo = []
for eps in epsilons:
    pandas,distorted_pandas, attack_maps = test(alexnet, eps, data)
    pandas_zoo.append(pandas)
    distorted_pandas_zoo.append(distorted_pandas)
    attack_maps_zoo.append(attack_maps)

succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
sad
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
succes
sad
succes
succes
succes
succes
succes
succes
sad
succes
succes
succes
succes
succes
succes
succes
succes


In [ ]:
#plt.imshow(torchvision.utils.make_grid( pandas_zoo[0][0].squeeze(), nrow=5).permute(1, 2, 0))

In [ ]:
plt.imshow(pandas_zoo[0][0][0].detach().permute(1,2,0).clamp(0,1))

In [ ]:
plt.imshow(distorted_pandas_zoo[0][0][0].detach().permute(1,2,0))

In [ ]:
plt.imshow(((attack_maps_zoo[0][0][0].permute(1,2,0))*80).clamp(0,1))

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
image = transforms.ToPILImage()(distorted_pandas_zoo[0][0][0])

image.save('panda_dis.jpg')

In [ ]:
panda = Image.open('panda_dis.jpg')

In [ ]:
tr = transforms.ToTensor()

In [ ]:
tr(panda)

In [ ]:
torch.max(alexnet(tr(panda).view(1,3,224,224)),dim=1)